In [1]:
import os
import warnings

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
from statsmodels.stats.multitest import multipletests
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from matplotlib.ticker import FormatStrFormatter
import re
from collections import defaultdict
from matplotlib.patches import Patch
from itertools import product

import misc
from misc import rename_cytokine, rename_CM, rename_cytokines, rename_CMs, rename_bloods, rename_factors, rename_Y, make_annot, convert_partial_year, rename_pathways, rename_Y_vars, get_day_suffix

import bcg_utils as utils
""
%load_ext autoreload
%autoreload 1
%aimport misc
%aimport bcg_utils
%aimport bcg_colors

from bcg_colors import *

_ = misc.setup_plotting(style='ticks', context='notebook')

In [2]:
CELLTYPE = 'PBMC'
RANK_METRIC = 'p.value'
TOP_N = 1000
ENR_PVAL = 0.05

In [3]:
DEBUG = False
SAVE_EXT_TABLES = False

# Suppl. Tables

In [4]:
df = misc.get_sample_annot()
peaks_df = misc.get_peak_annot()
table_name = ATAC_V1
for visit, Y_name, de_fig_n, enr_fig_n in [
    ('V1', 'CM', '3D', '3E'),
    ('V1', 'CYTO', '4E', '4F'),
    ('V2', 'CYTO', None, None),
    ('V3', 'CYTO', '6D', '6E')
]:
    # START
    suppl_tables = misc.get_container_with_tables(
        [ATAC_V1, ENRICH_V1, ENRICH_V1_PROMOTERS, ENRICH_V1_LOLA],
        modes=['a', 'a', 'a', 'a']
    )
    
    print(visit, Y_name)
    if Y_name == 'CYTO':
        proteins = df.columns[(df.columns.str.contains('^CYTO:.*_good$') & ~df.columns.str.contains('lactate'))]
    else:
        proteins = df.columns[df.columns.str.contains('^CM:')]

    if DEBUG:
        proteins = proteins[[-2, -7, -6]]
        
    de_suppl_dict = {}
    enr_suppl_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: {}))))
    
    for protein in proteins:
        coef = f'{f"LFC_{visit}_CORR_" if visit != "V1" else ""}{misc.safe_R_name(protein)}'
        model = f'V1.batch.sex.age{"" if Y_name == "CYTO" else ".bmi.oralContra"}.blood.TSS_enr.visit_time.{coef}'
        de_suppl_dict[protein] = misc.read_de(CELLTYPE, model, contrasts=coef)
        
        for region_filter in [misc.GENE_AND_DISTAL_10kb, misc.TSS_PROXIMAL]:
            for direction in [-1, 1]:
                enr_fn = misc.enrichr_fn(CELLTYPE, model, data='results', coef=coef, effect_size_filter=None, rank_metric=RANK_METRIC,
                                         top_n=TOP_N, direction=direction, db=None, regions=region_filter)
                enr_df = pd.read_csv(enr_fn, sep='\t', usecols=['Gene_set'] + get_ENR_suppl_cols())
                for library in [misc.GO_BIO_PROCESS, misc.KEGG]:
                    enr_suppl_dict['enrichr'][region_filter][library][direction][protein] = enr_df.loc[enr_df['Gene_set'] == library + '_min15_max500'].copy()
        
        for direction in [-1, 1]:
            enr_df = misc.read_enrichr_or_lola(CELLTYPE, model, coef=coef, effect_size_filter=None,
                                               rank_metric=RANK_METRIC, top_n=TOP_N, region_filter=None,
                                               method='lola', direction=direction)
            for library in [HOCOMOCO, EPI_ROADMAP]:
                library_mask = enr_df['Gene_set'] == library
                enr_df.loc[library_mask, 'Adjusted P-value'] = misc.adjusted_pvals(enr_df.loc[library_mask, 'P-value'], method='fdr_bh')
                enr_suppl_dict['lola'][None][library][direction][protein] = enr_df.loc[library_mask].reset_index().copy()
        del enr_df
        
    de_suppl_df = misc.make_suppl_table_atac_seq_d0_immune(de_suppl_dict, Y_name, proteins, visit,
                                                           peaks_df=peaks_df, region_filter=misc.GENE_AND_DISTAL_10kb,
                                                           limit_top_n=TOP_N, rank_metric=RANK_METRIC)
    del de_suppl_dict
    display(de_suppl_df.head())
    
    enr_suppl_dfs = defaultdict(lambda: {})
    for method in enr_suppl_dict:
        for region_filter in enr_suppl_dict[method]:
            enr_suppl_dfs[method][region_filter] = misc.make_suppl_table_enrich_d0_immune(enr_suppl_dict[method][region_filter], Y_name, proteins,
                                                                                          method, limit_pval=ENR_PVAL)
            display(enr_suppl_dfs[method][region_filter].head())
    del enr_suppl_dict
    
    cytokine_fold_changes = visit != 'V1' and Y_name == 'CYTO'
    # save selected regions to supplement; if cytokine_fold_changes, split by phenotype and subset to phenotype-related cytokines
    misc.suppl_table_atac_seq_d0_immune(de_suppl_df.loc[~de_suppl_df[DISTAL_MAP_COL].isnull()], Y_name, visit,
                                        split_by_phenotype=cytokine_fold_changes, split_by_protein=False,
                                        only_selected=True, phenotypes=['innate_nonspecific_24h_wo_LAC', 'adaptive_MTB_7d'],
                                        drop_padj=False, suppl_tables=suppl_tables, gene_sets=False, table_name=table_name, fig_n=de_fig_n)
    # save all gene sets to supplement; if cytokine_fold_changes, split by phenotype and subset to phenotype-related cytokines
    for method, region_filter, enr_table_name, fig_n in [
        ('enrichr', misc.GENE_AND_DISTAL_10kb, ENRICH_V1, enr_fig_n),
        ('enrichr', misc.TSS_PROXIMAL, ENRICH_V1_PROMOTERS, None),
        ('lola', None, ENRICH_V1_LOLA, None)
    ]: 
        misc.suppl_table_atac_seq_d0_immune(enr_suppl_dfs[method][region_filter], Y_name, visit,
                                            split_by_phenotype=cytokine_fold_changes, split_by_protein=False,
                                            only_selected=True, phenotypes=['innate_nonspecific_24h_wo_LAC', 'adaptive_MTB_7d'],
                                            promoters=region_filter == misc.TSS_PROXIMAL,
                                            method=method, drop_padj=True,
                                            suppl_tables=suppl_tables, gene_sets=True, table_name=enr_table_name, fig_n=fig_n)
    
    if SAVE_EXT_TABLES:
        for protein in proteins:
            protein = misc.rename_Y_vars(Y_name, protein, with_time=True)
            print(protein)
            protein_wo_spaces = protein.replace('(', '').replace(')', '').replace(
                ',', '').replace(' ', '_').replace('α', 'a').replace('β', 'b').replace('γ', 'g')
            ext_table_name = f'{protein_wo_spaces}_{NAMES["ATAC"]}_d0_{NAMES[Y_name]}_{misc.get_day_suffix(visit)}'
            ext_suppl_tables = misc.get_container_with_tables(ext_table_name, include_legends_sheet=False)
            for sheet_name, method, region_filter in [
                (None, None, None),
                ('geneSets', 'enrichr', misc.GENE_AND_DISTAL_10kb),
                ('geneSets_promoters', 'enrichr', misc.TSS_PROXIMAL),
                ('regionSets', 'lola', None)
            ]:
                if sheet_name is None:
                    assert method is None and region_filter is None
                    suppl_df = de_suppl_df
                else:
                    suppl_df = enr_suppl_dfs[method][region_filter]
                # select proteins
                suppl_df = suppl_df.loc[suppl_df.index.get_level_values(LONG_NAMES[Y_name]) == protein]

                misc.suppl_table_atac_seq_d0_immune(suppl_df, Y_name, visit,
                                                    split_by_phenotype=False, split_by_protein=False, only_selected=False,
                                                    promoters=region_filter == misc.TSS_PROXIMAL,
                                                    method=method, gene_sets=method is not None, drop_padj=True, force_sheet_name=sheet_name,
                                                    suppl_tables=ext_suppl_tables, table_name=ext_table_name, fig_n=de_fig_n[0] if de_fig_n else None, phenotypes=None)

        ext_suppl_tables[ext_table_name]['file'].close()
        misc.fix_font_for_suppl_tables(ext_suppl_tables, fix_legends_sheet=False)
        
    del enr_suppl_dfs
    del de_suppl_df
    
    # FINISH
    infos, tables = misc.close_tables_and_save(suppl_tables)
    for table, info_df in zip(tables, infos):
        print(table)
        display(info_df)  
    misc.fix_font_for_suppl_tables(suppl_tables)

Using this peak annotation: ../data/DE/peaks_filtered_PBMC.csv.gz
V1 CM


Coefficient  \
Region                   Inflammatory marker Phenotype selected                
chr5_132477463_132479163 CXCL10              NaN       False        0.152104   
                         CXCL9               NaN       False        0.119286   
chr4_1187003_1187698     CXCL9               NaN       True         0.170786   
chr18_61948784_61949609  CXCL9               NaN       True         0.230493   
chr20_50338395_50339509  CXCL10              NaN       False        0.176857   

                                                                  T-value  \
Region                   Inflammatory marker Phenotype selected             
chr5_132477463_132479163 CXCL10              NaN       False     7.462842   
                         CXCL9               NaN       False     6.554293   
chr4_1187003_1187698     CXCL9               NaN       True      5.887135   
chr18_61948784_61949609  CXCL9               NaN       True      5.745277   
chr20_50338395_50339509  CXCL10              NaN       False     5.739482   

                                                                      P-value  \
Region                   Inflammatory marker Phenotype selected                 
chr5_132477463_132479163 CXCL10              NaN       False     1.277508e-12   
                         CXCL9               NaN       False     2.992181e-10   
chr4_1187003_1187698     CXCL9               NaN       True      1.209636e-08   
chr18_61948784_61949609  CXCL9               NaN       True      2.561615e-08   
chr20_50338395_50339509  CXCL10              NaN       False     2.640937e-08   

                                                                 Adjusted P-value  \
Region                   Inflammatory marker Phenotype selected                     
chr5_132477463_132479163 CXCL10              NaN       False         8.184353e-08   
                         CXCL9               NaN       False         1.916941e-05   
chr4_1187003_1187698     CXCL9               NaN       True          3.874767e-04   
chr18_61948784_61949609  CXCL9               NaN       True          5.078470e-04   
chr20_50338395_50339509  CXCL10              NaN       False         8.459582e-04   

                                                                Promoter mapping  \
Region                   Inflammatory marker Phenotype selected                    
chr5_132477463_132479163 CXCL10              NaN       False                 NaN   
                         CXCL9               NaN       False                 NaN   
chr4_1187003_1187698     CXCL9               NaN       True                  NaN   
chr18_61948784_61949609  CXCL9               NaN       True                  NaN   
chr20_50338395_50339509  CXCL10              NaN       False                 NaN   

                                                                Distal mapping  
Region                   Inflammatory marker Phenotype selected                 
chr5_132477463_132479163 CXCL10              NaN       False               NaN  
                         CXCL9               NaN       False               NaN  
chr4_1187003_1187698     CXCL9               NaN       True              SPON2  
chr18_61948784_61949609  CXCL9               NaN       True               PIGN  
chr20_50338395_50339509  CXCL10              NaN       False               NaN

Odds ratio  \
Library                    Gene set                                           Inflammatory marker Phenotype selected Association               
KEGG_2019_Human            Herpes simplex virus 1 infection                   CASP-8              NaN       True     Positive       2.552041   
                           Autophagy                                          MMP-10              NaN       True     Positive       2.873498   
GO_Biological_Process_2018 myeloid cell differentiation (GO:0030099)          IL-6                NaN       True     Positive       4.720455   
                           erythrocyte differentiation (GO:0030218)           IL-6                NaN       True     Positive       5.386476   
                           negative regulation of lymphocyte proliferation... IL-17C              NaN       True     Negative       7.360731   

                                                                                                                                       P-value  \
Library                    Gene set                                           Inflammatory marker Phenotype selected Association                 
KEGG_2019_Human            Herpes simplex virus 1 infection                   CASP-8              NaN       True     Positive     1.221551e-07   
                           Autophagy                                          MMP-10              NaN       True     Positive     7.852634e-07   
GO_Biological_Process_2018 myeloid cell differentiation (GO:0030099)          IL-6                NaN       True     Positive     1.276133e-07   
                           erythrocyte differentiation (GO:0030218)           IL-6                NaN       True     Positive     2.431777e-07   
                           negative regulation of lymphocyte proliferation... IL-17C              NaN       True     Negative     1.326005e-07   

                                                                                                                                  Adjusted P-value  \
Library                    Gene set                                           Inflammatory marker Phenotype selected Association                     
KEGG_2019_Human            Herpes simplex virus 1 infection                   CASP-8              NaN       True     Positive             0.000033   
                           Autophagy                                          MMP-10              NaN       True     Positive             0.000212   
GO_Biological_Process_2018 myeloid cell differentiation (GO:0030099)          IL-6                NaN       True     Positive             0.000264   
                           erythrocyte differentiation (GO:0030218)           IL-6                NaN       True     Positive             0.000264   
                           negative regulation of lymphocyte proliferation... IL-17C              NaN       True     Negative             0.000291   

                                                                                                                                 Overlap  
Library                    Gene set                                           Inflammatory marker Phenotype selected Association          
KEGG_2019_Human            Herpes simplex virus 1 infection                   CASP-8              NaN       True     Positive     44/768  
                           Autophagy                                          MMP-10              NaN       True     Positive     31/481  
GO_Biological_Process_2018 myeloid cell differentiation (GO:0030099)          IL-6                NaN       True     Positive     19/186  
                           erythrocyte differentiation (GO:0030218)           IL-6                NaN       True     Positive     16/139  
                           negative regulation of lymphocyte proliferation... IL-17C              NaN       True     Negative      13/86

Odds ratio  \
Library         Gene set                         Inflammatory marker Phenotype selected Association               
KEGG_2019_Human Herpes simplex virus 1 infection SIRT2               NaN       True     Positive       3.326372   
                                                 TGF-β-1             NaN       True     Positive       3.166777   
                Systemic lupus erythematosus     TGF-β-1             NaN       True     Positive       7.139197   
                Herpes simplex virus 1 infection STAMPB              NaN       True     Positive       2.809011   
                                                 CXCL1               NaN       True     Positive       2.809011   

                                                                                                          P-value  \
Library         Gene set                         Inflammatory marker Phenotype selected Association                 
KEGG_2019_Human Herpes simplex virus 1 infection SIRT2               NaN       True     Positive     1.971546e-17   
                                                 TGF-β-1             NaN       True     Positive     6.967742e-16   
                Systemic lupus erythematosus     TGF-β-1             NaN       True     Positive     2.637827e-13   
                Herpes simplex virus 1 infection STAMPB              NaN       True     Positive     1.604806e-12   
                                                 CXCL1               NaN       True     Positive     1.604806e-12   

                                                                                                     Adjusted P-value  \
Library         Gene set                         Inflammatory marker Phenotype selected Association                     
KEGG_2019_Human Herpes simplex virus 1 infection SIRT2               NaN       True     Positive         5.283745e-15   
                                                 TGF-β-1             NaN       True     Positive         1.790710e-13   
                Systemic lupus erythematosus     TGF-β-1             NaN       True     Positive         3.389608e-11   
                Herpes simplex virus 1 infection STAMPB              NaN       True     Positive         4.252737e-10   
                                                 CXCL1               NaN       True     Positive         4.316929e-10   

                                                                                                    Overlap  
Library         Gene set                         Inflammatory marker Phenotype selected Association          
KEGG_2019_Human Herpes simplex virus 1 infection SIRT2               NaN       True     Positive     84/406  
                                                 TGF-β-1             NaN       True     Positive     81/406  
                Systemic lupus erythematosus     TGF-β-1             NaN       True     Positive      29/79  
                Herpes simplex virus 1 infection STAMPB              NaN       True     Positive     74/406  
                                                 CXCL1               NaN       True     Positive     74/406

Odds ratio  \
Library                Region set Inflammatory marker Phenotype selected Association               
Roadmap_Epigenomics_r9 H3K4me3    PD-L1               NaN       True     Negative       6.266783   
HOCOMOCO_v11           ZBT14      PD-L1               NaN       True     Negative       5.897173   
                       KAISO      PD-L1               NaN       True     Negative       5.387864   
Roadmap_Epigenomics_r9 H3K4me3    CDCP1               NaN       True     Positive       4.769605   
HOCOMOCO_v11           ZBT14      GDNF                NaN       True     Positive       4.873232   

                                                                                            P-value  \
Library                Region set Inflammatory marker Phenotype selected Association                  
Roadmap_Epigenomics_r9 H3K4me3    PD-L1               NaN       True     Negative     6.029318e-159   
HOCOMOCO_v11           ZBT14      PD-L1               NaN       True     Negative     1.184486e-142   
                       KAISO      PD-L1               NaN       True     Negative     4.821994e-124   
Roadmap_Epigenomics_r9 H3K4me3    CDCP1               NaN       True     Positive     3.974334e-122   
HOCOMOCO_v11           ZBT14      GDNF                NaN       True     Positive     2.407807e-119   

                                                                                      Adjusted P-value  \
Library                Region set Inflammatory marker Phenotype selected Association                     
Roadmap_Epigenomics_r9 H3K4me3    PD-L1               NaN       True     Negative        4.220523e-158   
HOCOMOCO_v11           ZBT14      PD-L1               NaN       True     Negative        4.749788e-140   
                       KAISO      PD-L1               NaN       True     Negative        9.668097e-122   
Roadmap_Epigenomics_r9 H3K4me3    CDCP1               NaN       True     Positive        2.782034e-121   
HOCOMOCO_v11           ZBT14      GDNF                NaN       True     Positive        9.655306e-117   

                                                                                        Overlap  
Library                Region set Inflammatory marker Phenotype selected Association             
Roadmap_Epigenomics_r9 H3K4me3    PD-L1               NaN       True     Negative     778/23396  
HOCOMOCO_v11           ZBT14      PD-L1               NaN       True     Negative     799/26192  
                       KAISO      PD-L1               NaN       True     Negative     812/28870  
Roadmap_Epigenomics_r9 H3K4me3    CDCP1               NaN       True     Positive     728/23396  
HOCOMOCO_v11           ZBT14      GDNF                NaN       True     Positive     767/26192

...

,Contents
Sheet name,
donorVar,Chromatin accessibility gene set enrichment of...
d0_age,Chromatin accessibility gene set enrichment of...
d0_sex,Chromatin accessibility gene set enrichment of...
d0_BMI,Chromatin accessibility gene set enrichment of...
d0_oralContraceptives,Chromatin accessibility gene set enrichment of...
d0_alcohol,Chromatin accessibility gene set enrichment of...
d0_sampleCollectionTime,Chromatin accessibility gene set enrichment of...
d0_inflaMark_d0,Chromatin accessibility gene set enrichment of...
d0_cytokine_d0,Chromatin accessibility gene set enrichment of...


Ext_S04.1_ATACseq_d0_geneSets_promoters


,Contents
Sheet name,
donorVar,Chromatin accessibility gene set enrichment of...
d0_age,Chromatin accessibility gene set enrichment of...
d0_sex,Chromatin accessibility gene set enrichment of...
d0_BMI,Chromatin accessibility gene set enrichment of...
d0_oralContraceptives,Chromatin accessibility gene set enrichment of...
d0_alcohol,Chromatin accessibility gene set enrichment of...
d0_sampleCollectionTime,Chromatin accessibility gene set enrichment of...
d0_inflaMark_d0,Chromatin accessibility gene set enrichment of...
d0_cytokine_d0,Chromatin accessibility gene set enrichment of...


Ext_S04.2_ATACseq_d0_regionSets


,Contents
Sheet name,
donorVar,Chromatin accessibility region set enrichment ...
d0_age,Chromatin accessibility region set enrichment ...
d0_sex,Chromatin accessibility region set enrichment ...
d0_BMI,Chromatin accessibility region set enrichment ...
d0_oralContraceptives,Chromatin accessibility region set enrichment ...
d0_alcohol,Chromatin accessibility region set enrichment ...
d0_sampleCollectionTime,Chromatin accessibility region set enrichment ...
d0_inflaMark_d0,Chromatin accessibility region set enrichment ...
d0_cytokine_d0,Chromatin accessibility region set enrichment ...
